# Week Five: Integration and differentiation

To be graded, your notebook must be runnable start to finish. If you can't make an in-notebook test pass, comment it out for to attempt to get partial credit. You should replace the `...` markers with your code. Do not change the names of the pre-defined ALL_CAPS variables and functions. (If you start from scratch, make sure you match the requested function names and requested ALL_CAPS variables). Other that that, you are free to define or make anything you wish. Remember that functions will often be tested with different numbers than the ones you are given.

In [1]:
# EID is your 6+2 UC Electronic ID
EID = '800159499'
NAME = 'Michael Moran'

In [1]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

# Problem 1: 2D Trapezoidal rule

You can extend the trapezoidal rule to 2D. I'll give you a few hints:

The weight matrix is:
$$
W_{ij} = \begin{matrix}
1 & 2 & 2 & \cdots & 2 & 2 & 1 \\
2 & 4 & 4 & \cdots & 4 & 4 & 2 \\
2 & 4 & 4 & \cdots & 4 & 4 & 2 \\
\vdots & \vdots & \vdots & \ddots & \vdots & \vdots & \vdots \\
2 & 4 & 4 & \cdots & 4 & 4 & 2 \\
2 & 4 & 4 & \cdots & 4 & 4 & 2 \\
1 & 2 & 2 & \cdots & 2 & 2 & 1
\end{matrix}
$$

If you use the trick I used in the slides instead of explicitly creating this matrix, you would split the sum into 4 parts. However, it's really quite easy to make the matrix above

The "bin" width $h$ is joined by the 2nd dimension bin width of $k$, and then the trapezoidal rule integral value is 

$$
I_\mathrm{tr} = \frac{1}{4} h k \sum_i \sum_j W_{ij} f(x_{i}, y_{j})
$$

Write a function that takes a 2-parameter function 'f' and integrates it with the given parameters (further explanations are built into the docstring of the function). I'm writing a Numpy-style docstring.

See also: <http://mathfaculty.fullerton.edu/mathews/n2003/SimpsonsRule2DMod.html>

In [39]:
def trap_2d(f, xrange, yrange, xN, yN):
    '''Integrate a 2D function with the trapezoidal rule.
    
    This takes a 2D function and integrates it over a 2D grid.
    This uses the Trapezoidal rule in 2D.
    
    Parameters
    ----------
    f : callable
        The function to integrate. The function should take an x and y value (possibly arrays).
    xrange : (float, float)
        The min and max values for x, as an array
    yrange : (float, float)
        The min and max values for y, as an array
    xN : int
        The number of evaluations along x
    yN : int
        The number of evaluations along x

    Returns
    -------
    float
        The total integral
    
    '''
    # If these are *not* arrays, convert them into arrays. Otherwise, leave them alone.
    # (If we used np.array, we'd needlessly make a copy if it aready was an array)
    xrange = np.asarray(xrange)
    yrange = np.asarray(yrange)
    
    # There are N-1 "bins"
    h = (xrange[1] - xrange[0])/(xN-1)
    k = (yrange[1] - yrange[0])/(yN-1)
    
    #creates weighted matrix
    weighted = np.full((len(xrange),len(yrange)), 4)
    weighted[0,0] = 1
    weighted[0,len(yrange)-1] = 1
    weighted[len(xrange)-1,0] = 1
    weighted[len(xrange)-1,len(yrange)-1] = 1
    weighted[1:-1,0] = 2
    weighted[0,1:-1] = 2
    weighted[1:-1,len(xrange)-1] = 2
    weighted[len(yrange)-1,1:-1] = 2

    #does the integration
    integra = (0.25 * xN * yN * 
              np.sum((weighted[i,j] * f(xrange,yrange) for i in range(len(xrange)) for j in range(len(yrange))
                      )))
    return integra

Feel free to add more tests, I'm adding the simplest one I can think of:

In [26]:
def f(x,y):
    return 1

In [40]:
area = trap_2d(f, [0, 10], [0, 10], 10, 10)
assert np.isclose(area, 100)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:48: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.


# Problem 2: MC integration

Write a function that performs MC integration of an unknown function of N dimensions. You can use `f(*array)` to call the function with the correct number of arguments (it expands to `f(array[0], array[1], ..., array[N])`). Return **two** values; the estimated integral and the estimated error on the integral.

In [21]:
def mc_integrate(f, N, samples, ranges):
    '''
    Integrate an ND function with a set of sampled MC points.
    
    Parameters
    ----------
    f : callable
        The function to integrate. The function should take N
        values (possibly equal length arrays).
    N : int
        The min and max values for x, as an array
    samples : int
        The number of samples to integrate over
    ranges : array[2, N]
        An array with the min and max for each dimension.

    Returns
    -------
    float
        The total integral
    float
        The estimated error = relative error * the total integral.
    '''
    
    ranges = np.asarray(ranges) # Just making sure ranges is an array
    
    # Hint: You can turn a 1D array, such as arr = array([a, b]) into a 2D array
    # with arr[:,np.newaxis] or arr[:,None] or arr.reshape(2,1) or arr.reshape(-1,1)
    
    # Our definition of ranges matches what you find in Scipy, and is natural to type,
    # but is a but less natural to use, for example:
    widths = ranges[:,1] - ranges[:,0]
    # empty list, fill it with N random points in ranges
    # take average of f
    
    vals = []
    for i in range(0,N):
      vals.append(np.random.uniform(ranges[i,0], ranges[i,1], samples))
    valsIntegra = np.array(vals)

    func = f(*valsIntegra)

    integra = np.mean(func) * (np.product(widths))
    err = integra/np.sqrt(samples)
    
    return integra, err

In [2]:
def f(x,y):
    return x + y

In [22]:
integral, error = mc_integrate(f, 2, 10_000, [[0,1],[0,1]])

In [23]:
print(integral, error)

0.998363695696666 0.009983636956966661


#### For comparison, we could use NQuad from Scipy:

In [24]:
from scipy import integrate

In [25]:
integrate.nquad(f, [[0,1],[0,1]])

(1.0, 1.662923778137264e-14)

# Problem 3: Second derivatives

Write a function that calculates:

$$
\frac{d^2 f(t)}{dt^2} \Biggr\rvert_\mathrm{cd}
=
\frac{
\left[
f(t + h) - f(t)
\right] - \left[
f(t) - f(t - h)
\right]
}{h^2}
$$

(The notes in class had a typo, fixed here.) Write a nice docstring as well.

In [ ]:
def d2fdt2(f, t, h):
  return (f(t+h)-f(t)-f(t)+f(t-h)) / h**2

Test your `d2fdt2` function on a function that you know the derivative for.

In [ ]:
print(-np.sin(1))
print(d2fdt2(np.sin, 1, 1e-5))

-0.8414709848078965
-0.8414702268311202
